In [1]:
from depth_kosmos import requests, DepthKosmosCaptioner
from depth_kosmos import Image as Image_PIL

depth_kosmos_captioner = DepthKosmosCaptioner()


Current directory: /root/Depth-SoM/Depth-Anything-V2


xFormers not available
xFormers not available
2025-03-10 12:12:23.302236: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-10 12:12:23.313529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741608743.332963    1052 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741608743.338277    1052 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 12:12:23.356344: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow 

Using device: cuda
Using device: cuda


In [2]:

url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTp5jMLHnfiO56w8iVWAwI4VvOu4B_5c2C1ww&s"
image = Image_PIL.open(requests.get(url, stream=True).raw)


In [3]:
# depth_kosmos_captioner.display_depth_images(image)
full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
print(full_caption_string)

Image loaded as NumPy array:
(183, 275, 3)


Closest: a woman carrying a basket of plants
----
Mid Range: a soldier
----
Farthest: a woman
----



In [ ]:
import os
import io
import pandas as pd
from PIL import Image as Image_PIL
import shutil
from datasets import load_dataset, Dataset, Image
from tqdm import tqdm
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file


# --- Configuration ---

hf_token = os.getenv("HF_TOKEN")

hf_user = "Rajarshi-Roy-research"
# dataset_name = "Flickr30k_Grounding_Som"
# commit_message = "Updating dataset with image paths"
# img_cols = ["image","wbox_image"]
# image_to_process = "image"

dataset_name = "COCO_OVSEG_Som"
commit_message = "Updating dataset with images"
img_cols = ["image"]
image_to_process = "image"

# --- Helper Functions ---
def save_image_from_bytes(image_data, idx, image_folder):
    """Saves an image from byte data and returns the saved file path."""
    try:
        image_bytes = image_data.get("bytes", None)
        if image_bytes:
            image = Image_PIL.open(io.BytesIO(image_bytes))
            image_path = os.path.join(image_folder, f"image_{idx}.jpg")
            image.save(image_path, format="JPEG")
            print(f"✅ Saved: {image_path}")
            return image_path
        else:
            print(f"⚠️ Skipped row {idx}: No bytes found")
            return None
    except Exception as e:
        print(f"❌ Error saving image at row {idx}: {e}")
        return None

def process_image_from_df(df, depth_kosmos_captioner, image_folder):
    """Processes the DataFrame, adding depth captions and saving progress."""
    if "depth_caption" not in df.columns:
        df["depth_caption"] = None

    for index, row in tqdm(df.iterrows(), total=len(df)):
        if row["depth_caption"] is None:
            try:
                if isinstance(row["image"], dict): #Handle case where image is still a dict
                    image_path = save_image_from_bytes(row["image"], index, image_folder)
                    if image_path:
                        image = Image_PIL.open(image_path)
                        full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
                        df.loc[index, "depth_caption"] = full_caption_string
                elif isinstance(row["image"], str): #Handle case where image is a file path
                    image = Image_PIL.open(row["image"])
                    full_caption_string = depth_kosmos_captioner.get_caption_with_depth(image)
                    df.loc[index, "depth_caption"] = full_caption_string
                else:
                    print(f"⚠️ Unexpected image type at index {index}: {type(row['image'])}")
                    df.loc[index, "depth_caption"] = "Error: Unexpected image type"

                df.to_parquet("df_with_captions.parquet")
            except Exception as e:
                print(f"Error processing image at index {index}: {e}")
                df.loc[index, "depth_caption"] = "Error"
                df.to_parquet("df_with_captions.parquet")

    return df

def push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=False):
    """Pushes the DataFrame to the Hugging Face Hub."""
    hf_dataset = Dataset.from_pandas(df)
    
    for img_col in img_cols:
        hf_dataset = hf_dataset.cast_column(img_col, Image())

    try:
        hf_dataset.push_to_hub(
            repo_id=f"{hf_user}/{dataset_name}",
            commit_message=commit_message,
            token=hf_token,
            private=private,
        )
        print(f"✅ Dataset successfully uploaded: {hf_user}/{dataset_name}")
    except Exception as e:
        print(f"❌ Error pushing to Hugging Face Hub: {e}")


# --- Main Script ---
# Clean up existing image folder
for img_col in img_cols:
    if os.path.exists(img_col):
        shutil.rmtree(img_col)
    os.makedirs(img_col)

# Load dataset
ds = load_dataset(f"{hf_user}/{dataset_name}")
df = ds["train"].to_pandas()

for img_col in img_cols:
    # Save images and update DataFrame
    for i, row in df.iterrows():
        if isinstance(row[img_col], dict):
            df.at[i, img_col] = save_image_from_bytes(row[img_col], i, img_col)

df.to_csv("updated_dataset.csv", index=False)
print("✅ Updated DataFrame saved as 'updated_dataset.csv'!")

#Process and add captions
try:
    df = pd.read_csv("updated_dataset.csv")
except FileNotFoundError:
    pass

df = process_image_from_df(df, depth_kosmos_captioner, image_to_process)

# Authenticate and push to Hugging Face

login(token=hf_token)
push_df_to_hugging_face(df, img_cols, hf_user, dataset_name, commit_message, hf_token, private=True)

df


README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/15.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/148 [00:00<?, ? examples/s]

✅ Saved: image/image_0.jpg
✅ Saved: image/image_1.jpg
✅ Saved: image/image_2.jpg
✅ Saved: image/image_3.jpg
✅ Saved: image/image_4.jpg
✅ Saved: image/image_5.jpg
✅ Saved: image/image_6.jpg
✅ Saved: image/image_7.jpg
✅ Saved: image/image_8.jpg
✅ Saved: image/image_9.jpg
✅ Saved: image/image_10.jpg
✅ Saved: image/image_11.jpg
✅ Saved: image/image_12.jpg
✅ Saved: image/image_13.jpg
✅ Saved: image/image_14.jpg
✅ Saved: image/image_15.jpg
✅ Saved: image/image_16.jpg
✅ Saved: image/image_17.jpg
✅ Saved: image/image_18.jpg
✅ Saved: image/image_19.jpg
✅ Saved: image/image_20.jpg
✅ Saved: image/image_21.jpg
✅ Saved: image/image_22.jpg
✅ Saved: image/image_23.jpg
✅ Saved: image/image_24.jpg
✅ Saved: image/image_25.jpg
✅ Saved: image/image_26.jpg
✅ Saved: image/image_27.jpg
✅ Saved: image/image_28.jpg
✅ Saved: image/image_29.jpg
✅ Saved: image/image_30.jpg
✅ Saved: image/image_31.jpg
✅ Saved: image/image_32.jpg
✅ Saved: image/image_33.jpg
✅ Saved: image/image_34.jpg
✅ Saved: image/image_35.jpg
✅ 

100%|██████████| 148/148 [03:47<00:00,  1.54s/it]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/148 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

✅ Dataset successfully uploaded: Rajarshi-Roy-research/COCO_OVSEG_Som


,id,image,depth_caption
0,000000023751_0,image/image_0.jpg,Closest: a person flying a kite\n----\nMid Ran...
1,000000028452_0,image/image_1.jpg,Closest: a pink shelf\n----\nMid Range: a pink...
2,000000013774_0,image/image_2.jpg,Closest: a person in a red suit\n----\nMid Ran...
3,000000018737_0,image/image_3.jpg,Closest: a pink camel\n----\nMid Range: a red ...
4,000000001000_15,image/image_4.jpg,Closest: a group of people\n----\nMid Range: t...
...,...,...,...
143,000000006471_5,image/image_143.jpg,Closest: a baseball player\n----\nMid Range: a...
144,000000013201_5,image/image_144.jpg,Closest: a skateboarder in mid-air\n----\nMid ...
145,000000001818_0,image/image_145.jpg,Closest: a pink and black striped fabric\n----...
146,000000002299_0,image/image_146.jpg,Closest: children\n----\nMid Range: a man\n---...
